In [1]:
from methods.widedta import _WideDTADataHandler
from modules.encoders import WideCNN


In [2]:
df = _WideDTADataHandler("Davis", path="..\data")

Found local copy...
Loading...
Done!
To log space...


Motif file loaded successfully.
Motif file up to date.


In [3]:
df.data["Target"]

0        (MKK, FFD, SRR, EQG, GSG, LGS, GSS, GGG, GST, ...
1        (PFW, KIL, NPL, LER, GTY, YYF, MGQ, QPG, KVL, ...
2        (MVL, GTV, LLP, PNS, YGR, DQD, TSL, CCL, CTE, ...
3        (MVD, GVM, ILP, VLI, MIA, LPS, PSM, EDE, KPK, ...
4        (MGA, AAK, LAF, AVF, LIS, CSS, GAI, LGR, SET, ...
                               ...                        
17286    (MGC, IKS, KEN, KSP, AIK, YRP, ENT, PEP, VST, ...
17287    (MAH, LRG, FAN, QHS, RVD, PEE, LFT, KLD, RIG, ...
17288    (MSS, MPK, PER, HAE, SLL, DIC, HDT, NSS, PTD, ...
17289    (MSS, LGA, SFV, QIK, FDD, LQF, FEN, CGG, GSF, ...
17290    (MPD, PAA, HLP, FFY, GSI, SRA, EAE, EHL, KLA, ...
Name: Target, Length: 17291, dtype: object

In [7]:
df.data["Motif"].str.len().max()

103

In [10]:
len(df.words_sets["Motif"])

2344